## Imports

In [ ]:
import os
import re
import sys
import typing
import gc
import pandas as pd
import pickle
import logging

sys.path.append(
    os.path.join('.','src')
)

from src.models import Pipeline
from src.nodes import *

In [ ]:
logging.basicConfig(filename='logs/logs.log', level=logging.DEBUG)

# Tweets Model

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_TWEETS.json', load_model_data = True)

In [ ]:
d = {
    'train_set_len' : len(pipeline.train_dataset),
    'train_set_tokens' : pipeline.train_dataset.token_len(),
    'val_set_len' : len(pipeline.val_dataset),
    'val_set_tokens' : pipeline.val_dataset.token_len(),
    'test_set_len' : len(pipeline.test_dataset),
    'test_set_tokens' : pipeline.test_dataset.token_len(),
}

In [ ]:
d

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline.train_model()

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_TWEETS.json', load_model_data = False)

In [ ]:
pipeline.load_model('models/tweets/tweets.pth')

In [ ]:
pipeline.generate(start_text = 'all', num_words = 40)

In [ ]:
pipeline.generate(start_text = 'what', num_words = 40)

In [ ]:
pipeline.generate(start_text = 'i like', num_words = 40)

# Wiki Model

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_WIKI.json', load_model_data = True)

In [ ]:
pipeline.perplexity(with_recall = True, with_tqdm = True)

In [ ]:
pipeline.load_model('models/wiki103/wiki103.pth')

In [ ]:
pipeline.perplexity(with_recall = True, with_tqdm = True)

In [ ]:
pipeline.train_model()

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline.generate(start_text = 'all', num_words=40)

In [ ]:
pipeline.generate(start_text = 'what', num_words=40)

In [ ]:
pipeline.generate(start_text = 'i like')

In [ ]:
d = {
    'train_set_len' : len(pipeline.train_dataset),
    'train_set_tokens' : pipeline.train_dataset.token_len(),
    'val_set_len' : len(pipeline.val_dataset),
    'val_set_tokens' : pipeline.val_dataset.token_len(),
    'test_set_len' : len(pipeline.test_dataset),
    'test_set_tokens' : pipeline.test_dataset.token_len(),
}

In [ ]:
d

## FedAVG

In [2]:
from src.federated_pipeline import Federated_AVG

import os
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
federated = Federated_AVG(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = True
)

100%|██████████| 5/5 [00:02<00:00,  2.38it/s]


In [4]:
federated.train(1, save_results = False)

100%|██████████| 5/5 [00:00<00:00, 2839.75it/s]


round 0


  0%|          | 0/5 [00:00<?, ?it/s]

round 1


100%|██████████| 5/5 [00:11<00:00,  2.32s/it]


In [5]:
pd.DataFrame(federated.results)

,0,1
perplexity,57.9413,85.4144
loss,4.07337,4.46081
f1_recall,0.344871,0.317658
f3_recall,0.454022,0.428005
generate,all former trump donors are reportedly barbara...,all points again for trump won what talking ab...
attack_perplexity,1038.57,825.814
perplexity_1,173.713,226.091
loss_1,5.15797,5.41972
f1_recall_1,0.194994,0.195563
f3_recall_1,0.305504,0.30778


In [6]:
federated.nodes[1].losses

{'total_loss': [5.318407286768374],
 'loss': [5.316334035733472],
 'reg_loss': [0.002073250146919315]}

## LICCHAVI

In [1]:
from src.federated_pipeline import Federated_LICCHAVI

import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
federated = Federated_LICCHAVI(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = True
)

100%|██████████| 5/5 [00:02<00:00,  2.40it/s]


In [3]:
federated.train(2, save_results = False)

  0%|          | 0/5 [00:00<?, ?it/s]

round 0


  0%|          | 0/5 [00:00<?, ?it/s]

round 1
tensor([334.4917], device='cuda:0', grad_fn=<AddBackward0>)
tensor([332.4322], device='cuda:0', grad_fn=<AddBackward0>)
tensor([330.3857], device='cuda:0', grad_fn=<AddBackward0>)
tensor([328.3524], device='cuda:0', grad_fn=<AddBackward0>)
tensor([326.3326], device='cuda:0', grad_fn=<AddBackward0>)
tensor([324.3262], device='cuda:0', grad_fn=<AddBackward0>)
tensor([322.3336], device='cuda:0', grad_fn=<AddBackward0>)
tensor([320.3547], device='cuda:0', grad_fn=<AddBackward0>)
tensor([318.3896], device='cuda:0', grad_fn=<AddBackward0>)
tensor([316.4385], device='cuda:0', grad_fn=<AddBackward0>)
tensor([314.5013], device='cuda:0', grad_fn=<AddBackward0>)
tensor([312.5781], device='cuda:0', grad_fn=<AddBackward0>)
tensor([310.6689], device='cuda:0', grad_fn=<AddBackward0>)
tensor([308.7738], device='cuda:0', grad_fn=<AddBackward0>)
tensor([306.8928], device='cuda:0', grad_fn=<AddBackward0>)
tensor([305.0257], device='cuda:0', grad_fn=<AddBackward0>)
tensor([303.1726], device='cuda:

tensor([136.2396], device='cuda:0', grad_fn=<AddBackward0>)
tensor([135.5490], device='cuda:0', grad_fn=<AddBackward0>)
tensor([134.8625], device='cuda:0', grad_fn=<AddBackward0>)
tensor([134.1800], device='cuda:0', grad_fn=<AddBackward0>)
tensor([133.5015], device='cuda:0', grad_fn=<AddBackward0>)
tensor([132.8270], device='cuda:0', grad_fn=<AddBackward0>)
tensor([132.1564], device='cuda:0', grad_fn=<AddBackward0>)
tensor([131.4898], device='cuda:0', grad_fn=<AddBackward0>)
tensor([130.8270], device='cuda:0', grad_fn=<AddBackward0>)
tensor([130.1682], device='cuda:0', grad_fn=<AddBackward0>)
tensor([129.5132], device='cuda:0', grad_fn=<AddBackward0>)
tensor([128.8620], device='cuda:0', grad_fn=<AddBackward0>)
tensor([128.2147], device='cuda:0', grad_fn=<AddBackward0>)
tensor([127.5711], device='cuda:0', grad_fn=<AddBackward0>)
tensor([126.9313], device='cuda:0', grad_fn=<AddBackward0>)
tensor([126.2952], device='cuda:0', grad_fn=<AddBackward0>)
tensor([125.6628], device='cuda:0', grad

...

tensor([334.3996], device='cuda:0', grad_fn=<AddBackward0>)
tensor([332.3390], device='cuda:0', grad_fn=<AddBackward0>)
tensor([330.2913], device='cuda:0', grad_fn=<AddBackward0>)
tensor([328.2569], device='cuda:0', grad_fn=<AddBackward0>)
tensor([326.2358], device='cuda:0', grad_fn=<AddBackward0>)
tensor([324.2283], device='cuda:0', grad_fn=<AddBackward0>)
tensor([322.2344], device='cuda:0', grad_fn=<AddBackward0>)
tensor([320.2542], device='cuda:0', grad_fn=<AddBackward0>)
tensor([318.2879], device='cuda:0', grad_fn=<AddBackward0>)
tensor([316.3355], device='cuda:0', grad_fn=<AddBackward0>)
tensor([314.3970], device='cuda:0', grad_fn=<AddBackward0>)
tensor([312.4725], device='cuda:0', grad_fn=<AddBackward0>)
tensor([310.5620], device='cuda:0', grad_fn=<AddBackward0>)
tensor([308.6656], device='cuda:0', grad_fn=<AddBackward0>)
tensor([306.7831], device='cuda:0', grad_fn=<AddBackward0>)
tensor([304.9147], device='cuda:0', grad_fn=<AddBackward0>)
tensor([303.0602], device='cuda:0', grad

tensor([129.9205], device='cuda:0', grad_fn=<AddBackward0>)
tensor([129.2649], device='cuda:0', grad_fn=<AddBackward0>)
tensor([128.6131], device='cuda:0', grad_fn=<AddBackward0>)
tensor([127.9651], device='cuda:0', grad_fn=<AddBackward0>)
tensor([127.3209], device='cuda:0', grad_fn=<AddBackward0>)
tensor([126.6805], device='cuda:0', grad_fn=<AddBackward0>)
tensor([126.0437], device='cuda:0', grad_fn=<AddBackward0>)
tensor([125.4107], device='cuda:0', grad_fn=<AddBackward0>)
tensor([124.7813], device='cuda:0', grad_fn=<AddBackward0>)
tensor([124.1556], device='cuda:0', grad_fn=<AddBackward0>)
tensor([123.5335], device='cuda:0', grad_fn=<AddBackward0>)
tensor([122.9150], device='cuda:0', grad_fn=<AddBackward0>)
tensor([122.3001], device='cuda:0', grad_fn=<AddBackward0>)
tensor([121.6888], device='cuda:0', grad_fn=<AddBackward0>)
tensor([121.0810], device='cuda:0', grad_fn=<AddBackward0>)
tensor([120.4767], device='cuda:0', grad_fn=<AddBackward0>)
tensor([119.8760], device='cuda:0', grad

tensor([64.7546], device='cuda:0', grad_fn=<AddBackward0>)
tensor([64.4937], device='cuda:0', grad_fn=<AddBackward0>)
tensor([64.2344], device='cuda:0', grad_fn=<AddBackward0>)
tensor([63.9768], device='cuda:0', grad_fn=<AddBackward0>)
tensor([63.7209], device='cuda:0', grad_fn=<AddBackward0>)
tensor([63.4667], device='cuda:0', grad_fn=<AddBackward0>)
tensor([63.2141], device='cuda:0', grad_fn=<AddBackward0>)
tensor([62.9632], device='cuda:0', grad_fn=<AddBackward0>)
tensor([62.7139], device='cuda:0', grad_fn=<AddBackward0>)
tensor([62.4662], device='cuda:0', grad_fn=<AddBackward0>)
tensor([62.2201], device='cuda:0', grad_fn=<AddBackward0>)
tensor([61.9756], device='cuda:0', grad_fn=<AddBackward0>)
tensor([61.7327], device='cuda:0', grad_fn=<AddBackward0>)
tensor([61.4914], device='cuda:0', grad_fn=<AddBackward0>)
tensor([61.2517], device='cuda:0', grad_fn=<AddBackward0>)
tensor([61.0135], device='cuda:0', grad_fn=<AddBackward0>)
tensor([60.7769], device='cuda:0', grad_fn=<AddBackward0

 20%|██        | 1/5 [00:04<00:18,  4.59s/it]


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(federated.results)

In [ ]:
federated.nodes[1].losses

In [ ]:
federated = Federated_LICCHAVI(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = True
)

In [ ]:
federated.train(1, save_results = False)

In [ ]:
pd.DataFrame(federated.results)

In [ ]:
federated.nodes[1].losses